In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [4]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [5]:
PREFIX = 'jun5'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun5/jun5_1_high_speed',
 'jun5/jun5_2_hide_demand',
 'jun5/jun5_3_high_speed',
 'jun5/jun5_3_hide_demand',
 'jun5/jun5_4_high_speed',
 'jun5/jun5_0_base',
 'jun5/jun5_4_hide_demand',
 'jun5/jun5_0_hide_demand',
 'jun5/jun5_1_hide_demand',
 'jun5/jun5_4_base',
 'jun5/jun5_2_base',
 'jun5/jun5_3_base',
 'jun5/jun5_0_high_speed',
 'jun5/jun5_1_base',
 'jun5/jun5_2_high_speed']

# For running the real dumb test case

In [6]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': sats_games
    },
])
print(len(pgrid))

15


In [7]:
# num_iters = 1_000_000
# for config in pgrid:
#     CONFIG = 'cfr_external' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000'      
#     dispatch.dispatch_experiments(CONFIG, cpus=4, base_job_name='jun2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [8]:
num_iters = 10_000_000
for config in pgrid:
    CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000'      
    dispatch.dispatch_experiments(CONFIG, cpus=4, base_job_name='jun5outcome', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 81834
Submitted batch job 81835
Submitted batch job 81836
Submitted batch job 81837
Submitted batch job 81838
Submitted batch job 81839
Submitted batch job 81840
Submitted batch job 81841
Submitted batch job 81842
Submitted batch job 81843
Submitted batch job 81844
Submitted batch job 81845
Submitted batch job 81846
Submitted batch job 81847
Submitted batch job 81848
Submitted batch job 81849
Submitted batch job 81850
Submitted batch job 81851
Submitted batch job 81852
Submitted batch job 81853
Submitted batch job 81854
Submitted batch job 81855
Submitted batch job 81856
Submitted batch job 81857
Submitted batch job 81858
Submitted batch job 81859
Submitted batch job 81860
Submitted batch job 81861
Submitted batch job 81862
Submitted batch job 81863
Submitted batch job 81864
Submitted batch job 81865
Submitted batch job 81866
Submitted batch job 81867
Submitted batch job 81868
Submitted batch job 81869
Submitted batch job 81870
Submitted batch job 81871
Submitted ba

In [11]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
             81838                           jun5_jun5_2_hide_demand-cfr_outcomemccfr_outcome-101_jun5outcome PD .      0:00      1 (Resources)
             81839                           jun5_jun5_2_hide_demand-cfr_outcomemccfr_outcome-102_jun5outcome PD .      0:00      1 (Priority)
             81840                            jun5_jun5_3_high_speed-cfr_outcomemccfr_outcome-100_jun5outcome PD .      0:00      1 (Priority)
             81841                            jun5_jun5_3_high_speed-cfr_outcomemccfr_outcome-101_jun5outcome PD .      0:00      1 (Priority)
             81842                            jun5_jun5_3_high_speed-cfr_outcomemccfr_outcome-102_jun5outcome PD .      0:00      1 (Priority)
             81843                           jun5_jun5_3_hide_demand-cfr_outcomemccfr_outcome-100_jun5outcome PD .      0:00      1 (Pr

In [47]:
!squeue | wc -l

91


In [ ]:
!squeue | grep "may30" | wc -l

In [21]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [13]:
Evaluation.objects.last()

<Evaluation: Evaluation for may30_may30_0_base-cfr_may20_23mccfr-100 (may30_sats) Iteration 999>